In [0]:
#mport cv2
#v2.version__

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
from IPython.display import Image

uploaded = files.upload()
img = cv.imread('man-of-steel.jpg')
#img = cv.imread('man-of-steel_small.jpg')
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)

uploaded = files.upload()
img_des = cv.imread('horrorposter-preview.jpg')
img_des_rgb = cv.cvtColor(img_des, cv.COLOR_BGR2RGB)
# the shape of this image
print(img_rgb.shape)
plt.imshow(img_rgb)
plt.show()

plt.imshow(img_des_rgb)
plt.show()

In [0]:
# A in AX=
MatrixA = np.zeros((6,6))
# B in AX=B
VectorB = np.zeros((6,1))
# Estimated X in AX=B
#EstimatedX = np.zeros((6,1))
# The original similarity transformation matrix
SimTranMax = np.zeros((3,3))


## image 1: The 4 corners of the input image: bigger input image: man-of-steel.jpg
## point 1
# top left
img1_pt1 = np.array([[0],[0]])
# top right
img1_pt2 = np.array([[812],[0]])
# bottom left
img1_pt3 = np.array([[0],[1200]])
# bottom right
img1_pt4 = np.array([[812],[1200]])


'''
## image 1: The 4 corners of the input image: bigger input image: man-of-steel_small.jpg
## point 1
# top left
img1_pt1 = np.array([[0],[0]])
# top right
img1_pt2 = np.array([[243],[0]])
# bottom left
img1_pt3 = np.array([[0],[360]])
# bottom right
img1_pt4 = np.array([[243],[360]])
'''


## image 2: The 4 corners of the Quadrilateral where the pixels inside would be replaced by the input pixels
# top left 
img2_pt1 = np.array([[377],[92]])
# top right
img2_pt2 = np.array([[780],[92]])
# bottom left
img2_pt3 = np.array([[377],[680]])
# bottom right
img2_pt4 = np.array([[780],[680]])

## fill in the coordinates to the AX=B
VectorB.itemset((0,0),img2_pt1.item(0,0))
VectorB.itemset((1,0),img2_pt2.item(0,0))
VectorB.itemset((2,0),img2_pt3.item(0,0))
#VectorB.itemset((3,0),img2_pt4.item(0,0))

VectorB.itemset((3,0),img2_pt1.item(1,0))
VectorB.itemset((4,0),img2_pt2.item(1,0))
VectorB.itemset((5,0),img2_pt3.item(1,0))
#VectorB.itemset((7,0),img2_pt4.item(1,0))
        
MatrixA.itemset((0,0),img1_pt1.item(0,0))
MatrixA.itemset((0,1),img1_pt1.item(1,0))
MatrixA.itemset((0,2),1.0)

MatrixA.itemset((1,0),img1_pt2.item(0,0))
MatrixA.itemset((1,1),img1_pt2.item(1,0))
MatrixA.itemset((1,2),1.0)

MatrixA.itemset((2,0),img1_pt3.item(0,0))
MatrixA.itemset((2,1),img1_pt3.item(1,0))
MatrixA.itemset((2,2),1.0)

MatrixA.itemset((3,3),img1_pt1.item(0,0))
MatrixA.itemset((3,4),img1_pt1.item(1,0))
MatrixA.itemset((3,5),1.0)

MatrixA.itemset((4,3),img1_pt2.item(0,0))
MatrixA.itemset((4,4),img1_pt2.item(1,0))
MatrixA.itemset((4,5),1.0)

MatrixA.itemset((5,3),img1_pt3.item(0,0))
MatrixA.itemset((5,4),img1_pt3.item(1,0))
MatrixA.itemset((5,5),1.0)


#print(MatrixA)

MatrixA_transpose = np.transpose(MatrixA)

#print(MatrixA_transpose)

At_A =  np.matmul(MatrixA_transpose,MatrixA)
At_A_inverse = np.linalg.inv(At_A)

#print(A_At)

#print(A_At_inverse.shape)
#print(VectorB.shape)

#EstimatedX =  A_At_inverse*MatrixA_transpose*VectorB
EstimatedX =  np.matmul(At_A_inverse,np.matmul(MatrixA_transpose,VectorB))

#print(EstimatedX.shape)

#print(EstimatedX)

# fill in the estimated X to similarity transformation matrix
SimTranMax.itemset((0,0),EstimatedX.item(0,0))
SimTranMax.itemset((0,1),EstimatedX.item(1,0))
SimTranMax.itemset((0,2),EstimatedX.item(2,0))
SimTranMax.itemset((1,0),EstimatedX.item(3,0))
SimTranMax.itemset((1,1),EstimatedX.item(4,0))
SimTranMax.itemset((1,2),EstimatedX.item(5,0))
SimTranMax.itemset((2,2),1.0) #this one is necessary

def CoordinateTransformation(SimTranMax,y,x):
    input_vector = np.array([[0],[0],[0]])
    output_vector = np.array([[0],[0],[0]])
    #print(type(output_vector))
    
    input_vector.itemset((0,0), x)
    input_vector.itemset((1,0), y)
    input_vector.itemset((2,0), 1.0)
    
    output_vector = np.matmul(SimTranMax,input_vector)
    
    output_vector.itemset((0,0), output_vector.item(0,0)/output_vector.item(2,0))
    output_vector.itemset((1,0), output_vector.item(1,0)/output_vector.item(2,0))
    return output_vector
  
  
  
def ImageTransformation(SimTranMax,src,result_img):
    
    h, w ,c = src.shape
    # output image
    #result_img = np.zeros_like(src)
    
    for i in range(h):
        for j in range(w):
        
            pixel_R = src.item(i,j,0)
            pixel_G = src.item(i,j,1)
            pixel_B = src.item(i,j,2)           
            
            output_vector = CoordinateTransformation(SimTranMax,i,j)
            
            output_vector = output_vector.astype(int)
            
            output_vector_x = output_vector.item(0,0)
            output_vector_y = output_vector.item(1,0)            
        
            result_img.itemset((output_vector_y,output_vector_x,0),pixel_R)
            result_img.itemset((output_vector_y,output_vector_x,1),pixel_G)
            result_img.itemset((output_vector_y,output_vector_x,2),pixel_B)
            
    return result_img
  
def InverseImageTransformation(SimTranMax_inv,src,result_img,img2_pt1,img2_pt2,img2_pt3,img2_pt4):
    
    h, w ,c = src.shape
    # output image
    #result_img = np.zeros_like(src)
    top_y =  img2_pt1.item(1,0)
    btm_y =  img2_pt4.item(1,0)
    left_x = img2_pt1.item(0,0)
    right_x =  img2_pt4.item(0,0)
    
    for i in range(top_y,btm_y):
        for j in range(left_x, right_x):
                                                                 
            output_vector = CoordinateTransformation(SimTranMax_inv,i,j)
            
            output_vector = output_vector.astype(int)
            
            output_vector_x = output_vector.item(0,0)
            output_vector_y = output_vector.item(1,0)            
            
            pixel_R = src.item(output_vector_y,output_vector_x,0)
            pixel_G = src.item(output_vector_y,output_vector_x,1)
            pixel_B = src.item(output_vector_y,output_vector_x,2)  
            
        
            result_img.itemset((i,j,0),pixel_R)
            result_img.itemset((i,j,1),pixel_G)
            result_img.itemset((i,j,2),pixel_B)
            
    return result_img

# 1st transformation:forward  
result_img = ImageTransformation(SimTranMax,img_rgb,img_des_rgb)

# 2nd transformation:backward
#SimTranMax_inv = np.linalg.inv(SimTranMax)
#result_img = InverseImageTransformation(SimTranMax_inv,img_rgb,img_des_rgb,img2_pt1,img2_pt2,img2_pt3,img2_pt4)

result_img_bgr = cv.cvtColor(result_img, cv.COLOR_RGB2BGR)

cv2_imshow(result_img_bgr)


  
  
  
  
  
  